In [14]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from sklearnex import patch_sklearn
patch_sklearn()

import heteroscedastic_uncertainty_regressor as hr


import sklearn

import plot_tools
import dataset_stats as ds

import pickle
import importlib

importlib.reload(hr)
importlib.reload(plot_tools)
importlib.reload(ds)

%matplotlib qt

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
VERSION = "v1"
MODEL_TYPE = "LOWER_BAND"

pdata_folder = os.path.abspath("./../processed_data/chorus_neural_network/")
rbsp_chorus_folder = os.path.join(pdata_folder, "observed_chorus")
output_folder = os.path.join(pdata_folder, "models", VERSION)
trained_model_folder = os.path.join(output_folder, "trained_models")

dataset = np.load(
    file=os.path.join(output_folder, rf"total_dataset_{VERSION}_{MODEL_TYPE}.npz")
)

X_conditional = dataset["X_conditional"]
X_convolutional = dataset["X_convolutional"]
y = dataset["y"]

print(y.shape)
print(X_conditional.shape)
print(X_convolutional.shape)

dataset.close()

t_data = X_conditional[:, -1]
X_conditional = X_conditional[:, :-1]

(3393325,)
(3393325, 7)
(3393325, 512)


In [3]:
# Preprocess dataset
RS_conditional = sklearn.preprocessing.RobustScaler(copy=False)
RS_conditional.fit(X_conditional)
transformed_X_conditional = RS_conditional.transform(X_conditional)

RS_convolutional = sklearn.preprocessing.RobustScaler(copy=False)
RS_convolutional.fit(X_convolutional)
transformed_X_convolutional = RS_convolutional.transform(X_convolutional)

In [5]:
saved_folder = os.path.abspath(trained_model_folder)

for m in range(1):
    model = hr.HeteroscedasticUncertaintyRegressor(input_dim=6, hidden_dim=384)
    model.load_state_dict(torch.load(os.path.join(saved_folder, f"model_{m}.pth"), weights_only=True))

    model.to("cuda")
    model.eval()
    # Evaluate and plot
    with torch.no_grad():

        t_X_cond_np = torch.from_numpy(transformed_X_conditional[0:10000, :]).to("cuda").type(torch.float32)
        t_X_conv_np = torch.from_numpy(transformed_X_convolutional[0:10000, np.newaxis, :]).to("cuda").type(torch.float32)
        pred_mean, pred_var = model(t_X_cond_np, t_X_conv_np)


    break

print(pred_mean, pred_var)

tensor([[1.5988],
        [1.0046],
        [1.2194],
        ...,
        [1.1289],
        [1.1444],
        [1.3439]], device='cuda:0') tensor([[2.7815],
        [0.2288],
        [0.1584],
        ...,
        [0.8060],
        [0.8600],
        [1.2271]], device='cuda:0')


In [8]:
print(np.array([(pred_mean[i], y[i]) for i in range(len(y))]))


IndexError: index 10000 is out of bounds for dimension 0 with size 10000

In [13]:
plt.errorbar(x=np.sqrt(y)[:10000].flatten(),
             y=pred_mean[:10000].cpu().numpy().flatten(),
             #yerr=np.sqrt(pred_var[:10000].cpu().numpy().flatten()),
             fmt=".")
plt.xscale("symlog")
plt.yscale("symlog")
plt.xlim(0, 1000)
plt.ylim(0, 1000)

(0.0, 1000.0)